## Draggable Label

`DraggableLabel` is a label that can be dragged and dropped to other fields.

In [1]:
from ipywidgets import Label, DraggableBox, DropBox, Textarea, VBox

In [2]:
def set_drag_data(box):
    box.drag_data['text/plain'] = box.children[0].value

def DraggableLabel(value, draggable=True):
    box = DraggableBox(Label(value))
    box.draggable = draggable
    return box

In [3]:
DraggableLabel("Hello Drag")

DraggableBox(child=Label(value='Hello Drag'))

You can drag this label anywhere (could be your shell etc.), but also to a text area:

In [4]:
Textarea()

Textarea(value='')

## `on_drop` handler

`DraggableLabel` can also become the drop zone (you can drop other stuff on it), if you implement the `on_drop` handler.

In [5]:
l1 = DraggableLabel("Drag me")
l1

DraggableBox(child=Label(value='Drag me'))

Now, drag this label on the label below.

In [6]:
l2 = DropBox(Label("Drop on me"))
def on_drop_handler(widget, data):
    """"Arguments:
    
    widget : widget class
        widget on which something was dropped
        
    data : dict
        extra data sent from the dragged widget"""
    print(data)
    text = data['text/plain']
    widget.child.value = "congrats, you dropped '{}'".format(text)

l2.on_drop(on_drop_handler)
l2

DropBox(child=Label(value='Drop on me'))

**Note** : You can also drop other stuff (like files, images, links, selections, etc). Try it out!

## Changing any widget into a drop zone with arbitrary drop operations

If you have more specific needs for the drop behaviour you can also use DropBox widgets, which implements `on_drop` handlers.

This DropBox will replace elements with text of the dropped element (works also for stuff which is not widget):

In [7]:
from ipywidgets import DropBox, Layout, Button

label = DraggableLabel("Drag me", draggable=True)
label

DraggableBox(child=Label(value='Drag me'))

or **Select and drag me!**

In [8]:
box = DropBox(Label("Drop here!"),
              layout=Layout(width='200px', height='100px'))
def on_drop(widget, data):
    text = data['text/plain']
    widget.child = Button(description=text.upper())

box.on_drop(on_drop)
box

DropBox(child=Label(value='Drop here!'), layout=Layout(height='100px', width='200px'))

## Adding widgets to container with a handler

You can also reproduce the Box example (adding elements to Box container) using `DropBox` with a custom handler:

In [9]:
from ipywidgets import DropBox, Layout, Label

label = DraggableLabel("Drag me", draggable=True)
label

DraggableBox(child=Label(value='Drag me'))

In [10]:
box = DropBox(VBox([Label('Drop here')]), 
              layout=Layout(width='200px', height='100px'))

def on_drop(widget, data):
    source = data['widget']
    widget.child.children += (source, )

box.on_drop(on_drop)
box

DropBox(child=VBox(children=(Label(value='Drop here'),)), layout=Layout(height='100px', width='200px'))

**Explanation**: Label widget sets data on the drop event of type `application/x-widget` that contains the widget id of the dragged widget.

## Setting custom data on the drop event

You can also set custom data on `DraggableLabel` that can be retrieved and used in `on_drop` event.

In [11]:
l = DraggableLabel("Drag me", draggable=True)
l.drag_data = {'application/custom-data' : 'Custom data'}
l

DraggableBox(child=Label(value='Drag me'), drag_data={'application/custom-data': 'Custom data'})

In [12]:
l2 = DropBox(Label("Drop here"))

def on_drop_handler(widget, data):
    """"Arguments:
    
    widget : widget class
        widget on which something was dropped
        
    data : dict
        extra data sent from the dragged widget"""
    
    text = data['text/plain']
    widget_id = data['widget'].model_id
    custom_data = data['application/custom-data']
    widget.child.value = ("you dropped widget ID '{}...' "
                    "with text '{}' and custom data '{}'"
                   ).format(widget_id[:5], text,  custom_data)

l2.on_drop(on_drop_handler)
l2

DropBox(child=Label(value='Drop here'))

## Making any widget draggable

`DraggableBox` can be used to wrap any widget so that it can be dragged and dropped.

In [13]:
from ipywidgets import DraggableBox, Button, VBox, Layout, IntSlider, Label

In [14]:
slider = IntSlider(layout=Layout(width='250px'), description='Drag me')
box = DraggableBox(slider, draggable=True)
box

DraggableBox(child=IntSlider(value=0, description='Drag me', layout=Layout(width='250px')))

In [15]:
box.drag_data

{}

In [16]:
def on_drop(*args):
    print(*args)

def attach_to_box(box, widget):
    box.children += (widget, )

In [17]:
vdropbox = DropBox(VBox([Label('Drop sliders below')], layout=Layout(width='350px', height='150px')))
# vdropbox.on_drop(lambda *args: attach_to_box(vdropbox.child, args[1]['widget']))
vdropbox.on_drop(on_drop)
vdropbox

DropBox(child=VBox(children=(Label(value='Drop sliders below'),), layout=Layout(height='150px', width='350px')…

## `on_dragenter` and `on_dragleave` handlers

In [18]:
button = Button(description="Drag me", button_style='info')
DraggableBox(button, draggable=True)

DraggableBox(child=Button(button_style='info', description='Drag me', style=ButtonStyle()))

In [19]:
button2 = Button(description="Drag me", button_style='warning')
DraggableBox(button2, draggable=True)

DraggableBox(child=Button(button_style='warning', description='Drag me', style=ButtonStyle()))

Now, drag this button on the label below.

In [22]:
label = "Hover on me"
l4 = DropBox(style={'background_color':"red"})

def on_dragenter_handler(widget):
    print("ENTER")
    l4.child.button_style = "success"
    

def on_dragleave_handler(widget):
    print("LEAVE")
    l4.child.button_style = "danger"


l4.on_dragenter(on_dragenter_handler)
l4.on_dragleave(on_dragleave_handler)
l4

DropBox(child=None)

## Draggable data columns

This implements a more complex example.

**Note**: You need to have bqplot installed for this example to work.

In [ ]:
import bqplot.pyplot as plt
from ipywidgets import Label, GridspecLayout, DropBox, Layout
import json

In [ ]:
def create_table(data):
    n_cols = len(data)
    n_rows = max(len(column) for column in data.values())
    grid = GridspecLayout(n_rows+1, n_cols)
    columnames = list(data.keys())
    for i in range(n_cols):
        column = columnames[i]
        data_json = json.dumps(data[column])
        grid[0, i] = DraggableLabel(columnames[i], draggable=True)
        grid[0, i].draggable = True
        grid[0, i].drag_data = {'data/app' : data_json}
        for j in range(n_rows):
            grid[j+1, i] = DraggableLabel(str(data[column][j]), draggable=True)
            grid[j+1, i].drag_data = {'data/app' : data_json}
    return grid

In [ ]:
box = DropBox(Label("Drag data from the table and drop it here."), layout=Layout(height='500px', width='800px'))
def box_ondrop(widget, data):
    fig = plt.figure()
    y = json.loads(data['data/app'])
    plt.plot(y)
    widget.child = fig
box.on_drop(box_ondrop)

In [ ]:
plot_data = {
    'col1': [ 4, 2,  1],
    'col2': [ 1, 3,  4],
    'col3': [-1, 2, -3]
}
create_table(plot_data)

You can drag the data by the labels or values (the whole column will be dragged).

In [ ]:
box

## Plot builder

Here is another example allowing to build plots from a list of labels.

In [17]:
from ipywidgets import SelectMultiple, Layout, DraggableBox, DropBox, HBox
import bqplot as bq

select_list = SelectMultiple(
    options=['Apples', 'Oranges', 'Pears'],
    value=['Oranges'],
    description='Fruits',
    disabled=False
)
select_box = DraggableBox(select_list, draggable=True)

fruits = {'Apples' : 5,
          'Oranges' : 1,
          'Pears': 3}


fig = bq.Figure(marks=[], fig_margin = dict(left=50, right=0, top=0, bottom=70))
fig.layout.height='300px'
fig.layout.width='300px'
fig_box = DropBox(fig)

fig2 = bq.Figure(marks=[], fig_margin = dict(left=50, right=0, top=0, bottom=70))
fig2.layout.height='300px'
fig2.layout.width='300px'
fig2_box = DropBox(fig2)

def on_fig_drop(widget, data):
    
    # get the figure widget
    fig = widget.child
    #get the selection widget
    selection_widget = data['widget'].child
    
    # get the selected fruits and prices
    selected = selection_widget.value
    prices = [fruits[f] for f in selected]
    
    sc_x = bq.OrdinalScale()
    sc_y = bq.LinearScale()

    ax_x = bq.Axis(scale=sc_x)
    ax_y = bq.Axis(scale=sc_y, orientation='vertical')
    bars = bq.Bars(x=selected, y=prices, scales={'x' : sc_x, 'y' : sc_y })
    fig.axes = [ax_x, ax_y]
    fig.marks = [bars]

fig_box.on_drop(on_fig_drop)
fig2_box.on_drop(on_fig_drop)

ModuleNotFoundError: No module named 'bqplot'

Select and drag fruits from the list to the boxes on the right. It's better to drag the selection using the "Fruits" label, due to a smalll glitch in the selection widget.

In [ ]:
HBox([select_box,
      fig_box,
      fig2_box])

## Dashboard builder

The drag and drop widgets can also be used to build a dashboard interactively.

First let's define the structure of the dashboard by using the `AppLayout` layout template widget.

In [89]:
from ipywidgets import AppLayout, Button, DropBox, Layout, VBox

Whenever a `DraggableBox` widget is dropped in a `Dropbox`, the content of the `Dropbox` will be replaced by the widget.

In [168]:
def attach_to_box(box, widget):
    box.child = widget


def create_expanded_dropbox(button_style):
    box = DropBox(Button(description='Drop widget here', button_style=button_style, layout=Layout(width='100%', height='100%')))
    box.on_drop(lambda *args: attach_to_box(box, args[1]['widget']))
    return box

Lets' create the app layout:

In [234]:
header = create_expanded_dropbox('success')
left_sidebar = create_expanded_dropbox('info')
center = create_expanded_dropbox('warning')
right_sidebar = create_expanded_dropbox('info')
footer = create_expanded_dropbox('success')

In [235]:
app = AppLayout(
    header=header,
    left_sidebar=left_sidebar,
    center=center,
    right_sidebar=right_sidebar,
    footer=footer
)
app

AppLayout(children=(DropBox(child=Button(button_style='success', description='Drop widget here', layout=Layout…

Now let's create the widgets to place in the dashboard.

In [231]:
from ipywidgets import DraggableBox, Dropdown, IntProgress, IntSlider, Label, Tab, Text, link

In [222]:
title = Label('My Custom Dashboard', layout=Layout(display='flex', justify_content='center', width='auto'))
DraggableBox(title, draggable=True)

DraggableBox(child=Label(value='My Custom Dashboard', layout=Layout(display='flex', justify_content='center', …

In [223]:
slider = IntSlider(min=0, max=10, step=1, layout=Layout(width='auto'), description='Slider')
DraggableBox(slider, draggable=True)

DraggableBox(child=IntSlider(value=0, description='Slider', layout=Layout(width='auto'), max=10))

In [224]:
progress = IntProgress(
    min=0,
    max=10,
    step=1,
    description='Loading:',
    orientation='horizontal',
    layout=Layout(width='auto')
)
link((slider, 'value'), (progress, 'value'))
DraggableBox(progress, draggable=True)

DraggableBox(child=IntProgress(value=0, description='Loading:', layout=Layout(width='auto'), max=10))

In [233]:
tab_contents = ['P0', 'P1', 'P2', 'P3', 'P4']
children = [Text(description=name) for name in tab_contents]
tab = Tab()
tab.children = children
for i in range(len(children)):
    tab.set_title(i, str(i))
DraggableBox(tab, draggable=True)

DraggableBox(child=Tab(children=(Text(value='', description='P0'), Text(value='', description='P1'), Text(valu…

Now let's drag the widgets and drop them in the app layout.

In JupyterLab you can open the widget in a different panel by right clicking on the `AppLayout` widget and selecting `Create New View for Output`:

![create-new-view-for-output](./images/create-new-view-for-output.png)